In [14]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re
import os

In [ ]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://transcripts.foreverdreaming.org/viewtopic.php?f=22&t=4411"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

In [3]:
item_links = [item.get_attribute("href") for item in driver.find_elements(By.CLASS_NAME, 'postlink')] # Get links to the scripts 

In [ ]:
# Scrape scripts from links 
for item_link in item_links[]:
    driver.get(item_link) # Click the link  
    time.sleep(3) 
    l = driver.find_elements(By.TAG_NAME, 'p')
    for i in l: 
        print (i.text) 


In [159]:
# Load scripts
scripts = [s for s in sorted(os.listdir('data')) if s.startswith('season')] # List of scripts sorted 

In [172]:
# Clean dataframes 
def prepare_dataframe(df):
    df.rename(columns = {list(df)[0]:'Line'}, inplace = True) 
    df['Character'] = df['Line'].str.split(' ').str[0] # Create a column with first word from line 
    df['Line'] = df['Line'].str.split(n=1).str[1] # Delete the first word from line 
    df = df[df['Character'].str.endswith(':', na=False)] # Delete row if doesn't end with ":" 

    # Change the order of columns 
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    # Capitalize character names 
    df['Character'] = df['Character'].str.capitalize()
    df['Character'] = df['Character'].str.replace(':', '')
    
    return df 

SyntaxError: invalid syntax (924040763.py, line 7)

In [164]:
# Create list of dataframes 
seasons_list = []
 
# Append seasons scripts into the list
for i in range(len(scripts)):
    df = pd.read_csv("data/"+scripts[i])
    seasons_list.append(df)


In [165]:
# Prepare dataframes 
season1 = prepare_dataframe(seasons_list[0])
season1['Season'] = '1'

season2 = prepare_dataframe(seasons_list[1])
season2['Season'] = '2'

season3 = prepare_dataframe(seasons_list[2])
season3['Season'] = '3'

season4 = prepare_dataframe(seasons_list[3])
season4['Season'] = '4'

season5 = prepare_dataframe(seasons_list[4])
season5['Season'] = '5'

season6 = prepare_dataframe(seasons_list[5])
season6['Season'] = '6'

season7 = prepare_dataframe(seasons_list[6])
season7['Season'] = '7'


In [166]:
# Merge dataframes
all_seasons = pd.concat([season1, season2, season3, season4, season5, season6, season7], ignore_index=True)
all_seasons

,Character,Line,Season
0,Lorelai,"Please, Luke. Please, please, please.",1
1,Luke,How many cups have you had this morning?,1
2,Lorelai,None.,1
3,Luke,Plus...,1
4,Lorelai,"Five, but yours is better.",1
...,...,...,...
134176,Rory,Well It wasn't very flattering.,7
134177,Lorelai,Well how will people know you're the world's g...,7
134178,Rory,I don't know.,7
134179,Lorelai,I guess they'll just have to read your stuff.,7


In [130]:
# Replace mispelled names 
all_seasons['Character'] = all_seasons['Character'].replace(['Loreai', 'Lorleai', 'Loreali', 'Miche', 'Sooki', 'Sookei', 'Christohper', 'Orelai',
                                                            'Richrad', 'Grandpa', 'Lorelail', 'Ory', 'Christoper', 'Lorlelai'], 
                                                            ['Lorelai', 'Lorelai', 'Lorelai', 'Michel', 'Sookie', 'Sookie', 'Christopher', 'Lorelai',
                                                            'Richard', 'Richard', 'Lorelai', 'Rory', 'Christopher', 'Lorelai'])

In [173]:
wrong = pd.DataFrame(all_seasons['Line'].str.startswith(":"))
wrong = wrong['Line'].sum()

In [131]:
# Save output to csv 
all_seasons.to_csv(r'/Users/julkakubisa/gilmore_girls_network/data/Gilmore_Girls_Lines.csv', index=False)